### Motivation:
what do we plan to do here

In [4]:
import os
import matplotlib.pyplot as plt
from itertools import combinations
import numpy as np
from PIL import Image as im
from PIL import Image
import shutil
import math
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
from sklearn import linear_model
import torch

torch.manual_seed(42)

#### Description:
Number of images (both in test and train folder) = 384 <br>
**MatchNumbersTable.npz** file contains number of matches between every pair of images
```python
    MatchNumbersTable.npz file size = (384,384)
    
    array([[   0.,  711., 1009., ...,  107.,  300.,  447.],
           [ 711.,    0.,  951., ...,  161.,  150.,  120.],
           [1008.,  950.,    0., ...,  199.,  484.,  572.],
           ...,
           [ 107.,  161.,  199., ...,    0.,  592.,  580.],
           [ 300.,  150.,  484., ...,  592.,    0., 1074.],
           [ 447.,  120.,  572., ...,  580., 1074.,    0.]])
```
***
- In **2D_list_of_all_npzs_with_class.npz** file
    - shape = ( 384 , 385 )
    - first 384 column = the path of npz file between the current image and every other images<br>
    - last column = class name of the current image
```python
    array([['',
        'Experiment2/All_npzs/10429168_D_lowres_10428972_D_lowres_matches.npz',
        'Experiment2/All_npzs/10429168_D_lowres_10429194_D_lowres_matches.npz',
        ...,
        'Experiment2/All_npzs/10429168_D_lowres_10429013_D_lowres_matches.npz',
        'Experiment2/All_npzs/10429168_D_lowres_10429016_D_lowres_matches.npz',
        'erato_petiverana'],
        ...,
        ['Experiment2/All_npzs/10429016_D_lowres_10429168_D_lowres_matches.npz',
        'Experiment2/All_npzs/10429016_D_lowres_10428972_D_lowres_matches.npz',
        'Experiment2/All_npzs/10429016_D_lowres_10429194_D_lowres_matches.npz',
        ...,
        'Experiment2/All_npzs/10429016_D_lowres_10429013_D_lowres_matches.npz',
        '', 'erato_notabilis']], dtype='<U100')
```
***
- **class_names** = set of all classes in the dataset
- **class_name_of_each_rowImg** = list of class name of every row image (384,1)
- **number_of_matches_between_every_image_with_className** = number of matches + class name of that image (384, 385)
           


In [5]:
#Create a table, 
#where each row contains information [match# with img1] [match# with img2] [match# with img384] [Class Name of this row]
path = 'Experiment2/MatchNumbersTable.npz'
number_of_matches_between_every_image = np.load(path)['arr_0']
TotalImage=number_of_matches_between_every_image.shape[0]

path = 'Experiment2/2D_list_of_all_npzs_with_class.npz'
name_of_npzfile_between_each_pair_with_className_of_rowImg = np.load(path)['arr_0']
class_name_of_each_rowImg=name_of_npzfile_between_each_pair_with_className_of_rowImg[:,TotalImage]

class_names=list(set(class_name_of_each_rowImg))
class_name_of_each_rowImg=np.reshape(class_name_of_each_rowImg,(TotalImage,1))
number_of_matches_between_every_image_with_className=np.append(number_of_matches_between_every_image,class_name_of_each_rowImg,1)



- **function create_dataset_wrt_k**
    - k = number of top matches for every class for the current image to consider
    - creates X 
        - size ( 384, 27) = (Number of images, Number of class)
        - per image, take average number of matches for every class
    - create Y
        - size (384, 1) = (Number of images, class name of the image)

In [6]:
def create_dataset_wrt_k(k_given,percentage,number_of_matches_between_every_image_with_className,class_names,class_name_of_each_rowImg):
    
    k=k_given
    X=[]
    Y=[]
    TotalImage=number_of_matches_between_every_image_with_className.shape[0]

    for mainImgNumber in range(TotalImage):
        x=[]
        for currentClassName in class_names:
            number_of_matches_per_class=[]

            for i in range(TotalImage):
                if i!= mainImgNumber and class_name_of_each_rowImg[i]==currentClassName:
                    number_of_matches_per_class.append(number_of_matches_between_every_image_with_className[mainImgNumber,i])

            number_of_matches_per_class=np.reshape(number_of_matches_per_class,(1,len(number_of_matches_per_class)))
            number_of_matches_per_class=number_of_matches_per_class.astype(np.float64)
            number_of_matches_per_class.sort(axis=1)
            
            if k>number_of_matches_per_class.shape[1]:
                k=len(number_of_matches_per_class)
            else:
                k=k_given
                
            if percentage==True:
                k=math.ceil(len(number_of_matches_per_class)*(k_given/100))
                    
            average_match_wrt_k=np.sum(number_of_matches_per_class[0][-k:])/k
            x.append(average_match_wrt_k)
            
        X.append(x)
        Y.append(number_of_matches_between_every_image_with_className[mainImgNumber,TotalImage])
    X=np.reshape(X,(TotalImage,len(class_names)))
    Y=np.reshape(Y,(TotalImage,1))
    #X=np.reshape(X,(1,len(class_names)))
    #for cn in range(len(class_names)):
        #print(class_names[cn]," = ",X[0][cn])
    #print(Y)
    
    return X,Y

In [7]:
[X,Y]=create_dataset_wrt_k(2,False,number_of_matches_between_every_image_with_className,class_names,class_name_of_each_rowImg)

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

Y_flatten=Y.flat

le.fit(Y_flatten)
list(le.classes_)

Y_transformed_int=le.transform(Y_flatten)
X=np.transpose(X)

#le.inverse_transform([2])

In [9]:
import torch
class LinearClassifier(torch.nn.Module):
    def __init__(self, input_dim=27, output_dim=27):
        super(LinearClassifier, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.linear(x)
        return x

#### Information about K-fold:
- we will have **K models** in K-fold algorithm
    - for each split of training set, train a new model on the training set and then test on testing data
    - take mean accuracy over k models
- leave one out cross validation (loocv) has low bias, high variance in real world data.

In [118]:
#LOOCV
def LOOCV(X,Y,le):
    y_t = torch.from_numpy(np.reshape(Y, (Y.shape[0],1)))
    X_t=torch.from_numpy(np.transpose(X))
    X_t = X_t.to(torch.float32)

    print("X = ",X_t.shape)
    print("Y = ",y_t.shape)


    avg_accuracy=0

    labels=[]
    x_axis=[]
    #unique_y=np.reshape(torch.unique(y_t), (torch.unique(y_t).shape[0],1))
    for i in range(X_t.shape[1]):
        labels.append(le.inverse_transform(y_t[i])[0])
        x_axis.append(i)

    #print(labels)

    class_wise_misclassification = [0]*27
    class_wise_n_images=[0]*27

    for i in range(X_t.shape[0]):
        X_train=torch.cat((X_t[0:i,:],X_t[i+1:,:]),0)
        X_test=X_t[i,:]
        y_train=torch.cat((y_t[0:i],y_t[i+1:]),0)
        y_test=y_t[i]

        model = LinearClassifier(2,27)
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
        accuracy=0

        #train on train data
        for epoch in range(1000):
            output = model(X_train)

            loss = criterion(output, y_train.view(-1))
            #print(loss.item())
            #all_loss.append(loss.item())
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()

        #prediction on test data
        result=model.forward(X_test)
        predicted=torch.argmax(result)

        class_wise_n_images[y_test]=class_wise_n_images[y_test]+1

        if predicted == y_test:
            accuracy+=1
        else:
            class_wise_misclassification[y_test]=class_wise_misclassification[y_test]+1
            #print("Error: predicted label: ",le.inverse_transform([predicted]), " Actual label: ",le.inverse_transform([y_test[0]]))

        avg_accuracy+=accuracy

    #plotLOOCV(X_t,class_wise_misclassification,class_wise_n_images,labels,x_axis)
    

    print("Accuracy: ",(avg_accuracy/Y.shape[0])*100," % ")
    #print(class_wise_misclassification)
    
    return (avg_accuracy/Y.shape[0])*100,class_wise_misclassification,class_wise_n_images


In [131]:
#create mimic pairs and run LOOCV
import csv
from tabulate import tabulate

table=[["Species_1","N_Image","Misclassified_Image","Weighted_error%","Species_2","N_Image","Misclassified_Image","Weighted_error%","Accuracy"]]
with open ('mimic_pairs.csv') as csv_file:
    csv_reader=csv.reader(csv_file,delimiter=",")
    for row in csv_reader:
        print(row[0],row[1])
        #print(le.transform([row[0]])[0])
        [X_m,Y_m]=create_mimic_data(X,Y_transformed_int,le.transform([row[0]])[0],le.transform([row[1]])[0])
        #print(X_m.shape, Y_m.shape)
        
        [accuracy,class_wise_misclassification,class_wise_n_images]=LOOCV(X_m,Y_m,le)
        
        
        row_to_add=[row[0],class_wise_n_images[le.transform([row[0]])[0]]\
                     ,class_wise_misclassification[le.transform([row[0]])[0]],\
                    (class_wise_misclassification[le.transform([row[0]])[0]]/class_wise_n_images[le.transform([row[0]])[0]])*100\
                    , row[1]\
                     ,class_wise_n_images[le.transform([row[1]])[0]],class_wise_misclassification[le.transform([row[1]])[0]],\
                     (class_wise_misclassification[le.transform([row[1]])[0]]/class_wise_n_images[le.transform([row[1]])[0]])*100\
                    ,accuracy]
        table.append(row_to_add)
    save_t=tabulate(table,headers='keys', tablefmt='firstrow', missingval='N/A')
    with open('table.txt', 'w') as f:
        f.write(save_t)
        
        
        

erato_petiverana melpomene_rosina_N
X =  torch.Size([53, 2])
Y =  torch.Size([53, 1])
Accuracy:  66.0377358490566  % 
erato_petiverana melpomene_rosina_S
X =  torch.Size([57, 2])
Y =  torch.Size([57, 1])
Accuracy:  64.91228070175438  % 
erato_hydara melpomene_melpomene
X =  torch.Size([53, 2])
Y =  torch.Size([53, 1])
Accuracy:  90.56603773584906  % 
erato_venus melpomene_vulcanus
X =  torch.Size([21, 2])
Y =  torch.Size([21, 1])
Accuracy:  61.904761904761905  % 
erato_cyrbia melpomene_cythera
X =  torch.Size([25, 2])
Y =  torch.Size([25, 1])
Accuracy:  76.0  % 
erato_notabilis melpomene_plesseni
X =  torch.Size([41, 2])
Y =  torch.Size([41, 1])
Accuracy:  65.85365853658537  % 
erato_etylus melpomene_ecuadorensis
X =  torch.Size([13, 2])
Y =  torch.Size([13, 1])
Accuracy:  69.23076923076923  % 
erato_emma melpomene_aglaope
X =  torch.Size([8, 2])
Y =  torch.Size([8, 1])
Accuracy:  50.0  % 
erato_favorinus melpomene_amaryllis
X =  torch.Size([12, 2])
Y =  torch.Size([12, 1])
Accuracy:  

In [84]:
def create_mimic_data(X,Y,name_1,name_2):
    X_m=[]
    Y_m=[]
    
    n_image=0
    
    for i in range(Y.shape[0]):
        if Y[i]== name_1 or Y[i]== name_2:
            n_image=n_image+1
            X_m.append(X[[name_1,name_2],i])
            Y_m.append(Y[i])
    X_m=np.reshape(X_m,(2,n_image))
    Y_m=np.reshape(Y_m,(n_image,1))
    return X_m,Y_m

In [42]:
def plotLOOCV(X_t,class_wise_misclassification,class_wise_n_images,labels,x_axis):
    plt.axis([0,X_t.shape[1], 0, 61])
    plt.bar(x_axis, class_wise_misclassification, color ='blue',width = 0.4)
    plt.xticks(x_axis, labels, rotation=90)
    plt.ylabel("Number of Images")
    plt.xlabel("Class Name")
    plt.title("Number of misclassified image per class for LOOCV")
    plt.show()

    plt.axis([0,X_t.shape[1], 0, 61])
    plt.bar(x_axis, class_wise_n_images, color ='green',width = 0.4)
    plt.xticks(x_axis, labels, rotation=90)
    plt.ylabel("Number of Images")
    plt.xlabel("Class Name")
    plt.title("Number of image per class")
    plt.show()

    weighted_error= [(i / j)*100 for i, j in zip(class_wise_misclassification, class_wise_n_images)]
    plt.axis([0,X_t.shape[1], 0, 100])
    plt.bar(x_axis, weighted_error, color ='maroon',width = 0.4)
    plt.xticks(x_axis, labels, rotation=90)
    plt.ylabel("Number of Images")
    plt.xlabel("Class Name")
    plt.title("weighted error (misclassified n_images /total n_images) per class")
    plt.show()

In [132]:
import torch
from torch.utils.data import Dataset

class CustomTensorDataset(Dataset):
    def __init__(self, dataset, transform_list=None):
        [data_X, data_y] = dataset
        #X_tensor, y_tensor = torch.tensor(data_X), torch.tensor(data_y)
        X_tensor, y_tensor= data_X.clone().detach(), data_y.clone().detach()
        tensors = (X_tensor, y_tensor)
        
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        
        self.tensors = tensors
        self.transforms = transform_list

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transforms:
            x = self.transforms(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)

In [133]:
#trainset = [X_train, y_train]
#testset = [X_test, y_test]
Data_with_label=[X_t,y_t]

transforms_list=None

if transforms_list:
    dataset= CustomTensorDataset(dataset=Data_with_label,transform_list=transforms_list_train)
else:
    dataset= CustomTensorDataset(dataset=Data_with_label)

In [134]:
print(len(dataset))
print(dataset[0])
print("Dataset : ",dataset[0][0])
print("Label : ",dataset[0][1])

384
(tensor([ 429.0000,  706.5000,  750.5000,  227.0000,  696.5000,  475.0000,
         803.0000,  373.5000,  282.5000,  970.0000,  551.5000, 1008.5000,
         282.0000,  461.0000,  659.5000,   65.0000,  183.0000,  639.0000,
         389.5000,  668.5000,  528.5000,  705.5000,  734.0000,  475.0000,
         301.0000,  375.5000,  439.0000]), tensor([12]))
Dataset :  tensor([ 429.0000,  706.5000,  750.5000,  227.0000,  696.5000,  475.0000,
         803.0000,  373.5000,  282.5000,  970.0000,  551.5000, 1008.5000,
         282.0000,  461.0000,  659.5000,   65.0000,  183.0000,  639.0000,
         389.5000,  668.5000,  528.5000,  705.5000,  734.0000,  475.0000,
         301.0000,  375.5000,  439.0000])
Label :  tensor([12])


In [13]:
def train_epoch(model,device,dataloader,loss_fn,optimizer):
    train_loss,train_correct=0.0,0
    model.train()
    
    for X_train,y_train in dataloader:
        X_train,y_train=X_train.to(device),y_train.to(device)
        #print(X_train.shape)
        optimizer.zero_grad()
        output=model(X_train)
        
        loss=loss_fn(output, y_train.view(-1))
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * X_train.size(0)
        predictions = torch.argmax(output,axis=1)
        train_correct += (predictions == y_train.view(-1)).sum().item()

    return train_loss,train_correct
  
def valid_epoch(model,device,dataloader,loss_fn):
    valid_loss, val_correct = 0.0, 0
    model.eval()
    
    for x_test, y_test in dataloader:
        
        x_test,y_test = x_test.to(device),y_test.to(device)
        output = model(x_test)
        
        loss=loss_fn(output,y_test.view(-1))
        
        valid_loss+=loss.item()*x_test.size(0)
        predictions = torch.argmax(output,axis=1)
        val_correct+=(predictions == y_test.view(-1)).sum().item()

    return valid_loss,val_correct

In [89]:
#K-fold
from sklearn.model_selection import KFold

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

criterion = torch.nn.CrossEntropyLoss()

num_epochs=100000
batch_size=384
k=15
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}

In [ ]:
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset

history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    
    model = LinearClassifier()
    model.to(device)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    previous_test_acc=0
    
    for epoch in range(num_epochs):
        train_loss, train_correct=train_epoch(model,device,train_loader,criterion,optimizer)
        #print(train_loss / len(train_loader.sampler)," : ",train_correct)
        test_loss, test_correct=valid_epoch(model,device,test_loader,criterion)
        
        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        test_loss = test_loss / len(test_loader.sampler)
        test_acc = test_correct / len(test_loader.sampler) * 100
            

        if epoch%100 ==0:
            #print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,num_epochs,train_loss,test_loss,train_acc,test_acc))
            print("Epoch:{}/{} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,num_epochs,train_acc,test_acc))
            if previous_test_acc == test_acc:
                #previous_test_acc=previous_test_acc
                break
            else:
                previous_test_acc=test_acc
            
            
    history['train_loss'].append(train_loss)
    history['test_loss'].append(test_loss)
    history['train_acc'].append(train_acc)
    history['test_acc'].append(test_acc) 

Fold 1
Epoch:1/100000 AVG Training Acc 0.56 % AVG Test Acc 7.69 %
Epoch:101/100000 AVG Training Acc 82.68 % AVG Test Acc 57.69 %
Epoch:201/100000 AVG Training Acc 93.30 % AVG Test Acc 65.38 %
Epoch:301/100000 AVG Training Acc 94.41 % AVG Test Acc 76.92 %
Epoch:401/100000 AVG Training Acc 97.21 % AVG Test Acc 80.77 %
Epoch:501/100000 AVG Training Acc 97.77 % AVG Test Acc 76.92 %
Epoch:601/100000 AVG Training Acc 97.49 % AVG Test Acc 80.77 %
Epoch:701/100000 AVG Training Acc 98.60 % AVG Test Acc 76.92 %
Epoch:801/100000 AVG Training Acc 99.16 % AVG Test Acc 76.92 %
Fold 2
Epoch:1/100000 AVG Training Acc 1.12 % AVG Test Acc 26.92 %
Epoch:101/100000 AVG Training Acc 85.75 % AVG Test Acc 80.77 %
Epoch:201/100000 AVG Training Acc 92.46 % AVG Test Acc 80.77 %
Fold 3
Epoch:1/100000 AVG Training Acc 0.00 % AVG Test Acc 19.23 %
Epoch:101/100000 AVG Training Acc 79.61 % AVG Test Acc 65.38 %
Epoch:201/100000 AVG Training Acc 92.74 % AVG Test Acc 88.46 %
Epoch:301/100000 AVG Training Acc 97.49 % AV

In [86]:
print("Avg Test Accuracy : ",mean(history['test_acc']) )
print("Avg Train Accuracy : ",mean(history['train_acc']) )

Avg Test Accuracy :  84.92577597840756
Avg Train Accuracy :  98.17751528859847


In [39]:
history['test_loss']

[196.34608459472656,
 163.61392211914062,
 198.8260955810547,
 74.64879608154297,
 91.2473373413086]

#### K fold
- for k=10, epoch = 100000, batch size=128
    - Avg Test Accuracy :  87.28070175438597
    - Avg Train Accuracy :  98.17701264974448
- for k=5, epoch = 100000, batch size = 384
    - Avg Test Accuracy :  84.11825017088175
    - Avg Train Accuracy :  98.56825584838614
- for k=5, epoch = 100000, batch size= 128
    - Avg Test Accuracy :  87.23171565276827
    - Avg Train Accuracy :  99.74025974025975
- for k=10, epoch = 100000, batch size= 64
    - Avg Test Accuracy :  87.01754385964912
    - Avg Train Accuracy :  99.24855491329478
- for k=5, epoch = 100000, batch size= 248
    - Avg Test Accuracy :  84.11483253588517
    - Avg Train Accuracy :  98.69876052286476
- for k=10, epoch = 100000, batch size= 248
    - Avg Test Accuracy :  87.53036437246963
    - Avg Train Accuracy :  98.00251319426992